In [ ]:
import geopandas as gpd
import geoparquet
import pyarrow as pq

In [ ]:
dataset = gpd.read_parquet(url)
url = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/geoparquet-projected/BE_VLG_2021_EC21.parquet"

In [ ]:
dataset.plot()

In [ ]:
dataset.explore()


In [ ]:
dataset.head()

In [ ]:
dataset.columns

In [ ]:
dataset.geometry

In [ ]:
import fused

udf = fused.load(
    "https://github.com/fusedio/udfs/tree/main/public/Overture_Maps_Example"
)

gdf = fused.run(udf, x=2622, y=6333, z=14)
gdf.plot()

### More data

In [ ]:
url_DE = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/geoparquet-projected/DE_LS_2021_EC21.parquet"
url_EE = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/geoparquet-projected/EE_2021_EC21.parquet"
url_ES = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/geoparquet-projected/ES_NA_2020_EC21.parquet"
url_RO = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/geoparquet-projected/RO_ny_EC21.parquet"
url_BE = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/geoparquet-projected/BE_VLG_2021_EC21.parquet"

In [ ]:
data_DE = gpd.read_parquet(url_DE)
data_EE = gpd.read_parquet(url_EE)
data_ES = gpd.read_parquet(url_ES)
data_RO = gpd.read_parquet(url_RO)
data_BE = gpd.read_parquet(url_BE)

In [ ]:
print(data_DE.columns)
print(data_EE.columns)
print(data_ES.columns)
print(data_RO.columns)
print(data_BE.columns)

### Data Dictionary

- geometry, country
- EC_hcat_n
- EC_hcat_c
- EC_trans_n
- EC_NUTS3

### Duckdb

In [ ]:
import duckdb

In [ ]:
# con = duckdb.connect()
# df = con.execute(f"SELECT * from read_parquet('{url_BE}');").fetchdf()

In [ ]:
df_BE = duckdb.read_parquet(url_BE)

In [ ]:
duckdb.sql("SELECT distinct EC_trans_n from df_BE limit 5").fetchall()

In [ ]:
duckdb.sql("SELECT distinct EC_hcat_n from df_BE limit 5").fetchall()

In [ ]:
duckdb.sql("SELECT distinct EC_hcat_c from df_BE limit 5").fetchall()

### All data

All data are available as a duckdb file. Can I import it from a cloud URI? 

Notes:
- won't work with duckdb==1.4.2 because url_eurocrops was created with outdated version and newer version are not
  backward compatible
  "IOException: IO Error: Trying to read a database file with version number 51, but we can only read versions between 64 and 67.
   The database file was created with DuckDB version v0.8.0 or v0.8.1."

In [ ]:
import duckdb

In [ ]:
!pip3 freeze | grep duckdb 

In [ ]:
url_eurocrops = "s3://us-west-2.opendata.source.coop/cholmes/eurocrops/eurocrops-full.duckdb"
df = duckdb.connect(":memory")
con.execute(f"""
INSTALL httpfs;
LOAD httpfs;
INSTALL aws;
LOAD aws;
SET s3_region='us-west-2';

""")

# con.execute(f"""
# ATTACH '{url_eurocrops}' as eurocrops (READ_ONLY); 
# """)

# , STORAGE_VERSION 'v0.8.1'

# eurocrops = con.execute(f"""
#     SELECT * FROM read_parquet('{url_eurocrops}')
# """).df()



In [ ]:
duckdb.sql("""
    SELECT * FROM read_parquet(
        's3://us-west-2.opendata.source.coop/cholmes/eurocrops/eurocrops-full.duckdb'
    )
""").df()

In [ ]:
# eurocrops = con.read_parquet(f"""
#     SELECT * FROM '{url_eurocrops}'
# """).df()

In [ ]:
con = duckdb.connect(database=url_eurocrops, read_only=True)
